# 序列資料預測 - RNN (DIY資料集)

## 1.DIY Dataset 序列資料
建立序列 Dataset，以等差數列，長度取 12
例如：x=[15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 125] y=135

### 1.1 DIY 製作『數字序列』資料之準備工作

In [ ]:
# 定義DIY函式
from numpy import array

# 傳入序列內容 raw, 指定每組序列長度 seq
def DIY(raw, seq):
	x, y = list(), list()
	for i in range(len(raw)):
		end = i + seq  # 設定序列結束點
		if end > len(raw)-1:  # 檢查是否結束序列
			break
		# 產生序列(x數據/題目 ,y標籤/答案)入
		seq_x, seq_y = raw[i:end], raw[end]
		x.append(seq_x)  # 元素加入list
		y.append(seq_y)  # 元素加入list
	return array(x), array(y)  # list轉矩陣 並傳回

# 自訂顯示函數
def show(x, y):
  for i in range(len(x)):
    print(x[i], y[i])  # 同時列出 x, y

### 1.2 原始數據

In [ ]:
# 設定序列內容
raw_data = []
for i in range(15,550,10):
    raw_data.append(i)

# 序列長度 seq
seq = 12

### 1.3 建立訓練集 X_train, Y_train

In [ ]:
# 使用函式 DIY() 及 show()
x_train, y_train = DIY(raw_data, seq)
show(x_train, y_train)

### 1.4 建立測試集 x_test, y_test

In [ ]:
# 使用未知數字建立測試集
test_data = []
for i in range(710,950,10): # x_train 15,550
    test_data.append(i)

# 序列長度 seq
seq = 12
x_test, y_test = DIY(test_data, seq)
show(x_test, y_test)

### 1.5 檢視 DIY Dataset 資料筆數

In [ ]:
# 查看資料結構
print(x_train.shape , x_test.shape)
print(y_train.shape , y_test.shape)

In [ ]:
# RNN 模型需要`三維矩陣`輸入數據
# np-2D 轉 3D 改變矩陣形狀 reshape
import numpy as np  #  2D, (3D-1, 3D-2, 3D-3為)

x_train_3d = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test_3d = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print(x_train_3d.shape)
print(x_test_3d.shape)
# print(x_train_3d)
# print(x_test_3d)

## 2.建模與訓練參數調整

### 2.1 匯入TensorFlow

In [ ]:
# 匯入 TensorFlow
import tensorflow

# 匯入建模所需的模組
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model     # 模型工具
from tensorflow.keras.models import Sequential  # 順序序模型
from tensorflow.keras.layers import Dense    # 全連接層
from tensorflow.keras import backend       # 後端模組

# RNN 循環神經網絡
from tensorflow.keras.layers import SimpleRNN

### 2.2 建模與編譯

In [ ]:
# 使用SimpleRNN建立RNN模型
model = Sequential(name='RNN')
model.add(SimpleRNN(units=6, input_dim=seq, activation='relu', name='RNN')) # 6個神經元
model.add(Dense(1)) #輸出層
model.summary()

# 編譯模型
model.compile(loss='mean_squared_error', optimizer='rmsprop')


In [ ]:
# 圖形顯示模型
from tensorflow.keras.utils import plot_model

# plot_model(model)
plot_model(model, show_shapes=True)
# plot_model(model, show_shapes=True, to_file='model.png')

### 2.3 調整模型 "訓練" 參數
- 傳回 model.fit() 的歷史記錄

In [ ]:
# 開始訓練
%%time
history = model.fit(x_train_3d, y_train, epochs=2000, verbose=0)


In [ ]:
# 查看
print(history.history)
print(history.history.keys())
print(len(history.history['loss']))

### 2.4 顯示訓練成效loss

In [ ]:
# 評估模型性能
train_result_RNN = model.evaluate(x_train_3d, y_train)
test_result_RNN = model.evaluate(x_test_3d, y_test)
print('RNN Train loss: {:.6f}'.format(train_result_RNN))
print('RNN Test loss: {:.6f}'.format(test_result_RNN))


### 2.5 查看訓練的過程

In [ ]:
# 顯示圖表來分析模型的訓練過程
import matplotlib.pyplot as plt

# 顯示訓練和驗證損失
loss = history.history['loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


### 2.6 重新建模前先刪除舊模組
- 重覆執行 model.fit() 是繼續之前，再增加訓練
- 繼續訓練：重覆執行訓練 model.fit()，查看每次的結果
- 比較重新建模後再執行的結果

In [ ]:
# 調整的參數與訓練成效，會累積影響現有模型
# 清除與刪除現有模型，調整參數才有意義

# backend.clear_session()
# del model

## 3.模型預測

In [ ]:
# 使用測試集預測
y_rnn = model.predict(x_test_3d, verbose=1)
#觀察: 預測值vs標籤
show(y_test, y_rnn) #自定義函式
